In [7]:
import pandas as pd
import os

event_df = pd.read_csv('../../Our Datasets/processed_recoveries_and_interceptions_dynamic_events.csv')

match_ids = event_df['source_file'].unique()

In [8]:
event_df = event_df.rename(columns={'Unique ID': 'unique_poss_id'})

event_df['source_file'] = event_df['source_file'].astype(str)

This should take 7 minutes roughly to run

In [9]:
base_dir = "../../data/matches"
all_tracking = []
for mid in match_ids:
    folder = os.path.join(base_dir, str(mid))
    files = os.listdir(folder)
    track_files = [f for f in files if "tracking_extrapolated" in f.lower()]
    if not track_files:
        print(f"❌ No tracking_extrapolation file for match {mid}")
        continue
    file_path = os.path.join(folder, track_files[0])
    print(f"📄 Reading JSONL: {file_path}")
    tracking_df_temp = pd.read_json(file_path, lines=True)
    tracking_df_temp["match_id"] = mid
    tracking_df_temp['match_id'] = tracking_df_temp['match_id'].astype(str)
    tracking_df_temp['frame'] = tracking_df_temp['frame'].astype(int)
    merged = tracking_df_temp.merge(
        event_df[['source_file', 'unique_poss_id', 'possession_start', 'possession_end']],
        left_on='match_id',
        right_on='source_file',
        how='left'
    )
    filtered_tracking = merged[(merged['frame'] == merged['possession_start'])].copy()
    filtered_tracking = pd.json_normalize(
        filtered_tracking.to_dict("records"),
        "player_data",
        ["frame", "timestamp", "period", "possession", "ball_data"]
    )
    filtered_tracking["possession_player_id"] = filtered_tracking["possession"].apply(lambda x: x.get("player_id"))
    filtered_tracking["possession_group"] = filtered_tracking["possession"].apply(lambda x: x.get("group"))
    filtered_tracking[["ball_x", "ball_y", "ball_z", "is_detected_ball"]] = pd.json_normalize(filtered_tracking.ball_data)
    filtered_tracking["match_id"] = mid
    filtered_tracking['match_id'] = filtered_tracking['match_id'].astype(str)
    all_tracking.append(filtered_tracking)
tracking_all = pd.concat(all_tracking, ignore_index=True)
tracking_all = tracking_all.drop(columns=["possession", "ball_data"])

In [10]:
tracking_all

In [11]:
tracking_all.to_csv('../../Our Datasets/processed_tracking_data_start.csv', index=False)